# Model Evaluation Metrics Demo

This notebook demonstrates how to evaluate and compare different NL-to-SQL models using various metrics.

In [ ]:
# Setup
import sys
import os
sys.path.append('..')

from app.metrics import ModelEvaluator
from app.inference import NL2SQLInference
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Initialize Evaluator

In [ ]:
# Initialize model evaluator
evaluator = ModelEvaluator()
print("✅ Model evaluator initialized")

## Create Test Dataset

In [ ]:
# Create test dataset
test_data = evaluator.create_test_dataset()

print(f"📊 Created test dataset with {len(test_data)} test cases:")
print()

for i, test_case in enumerate(test_data, 1):
    print(f"{i}. Question: {test_case['question']}")
    print(f"   Expected SQL: {test_case['expected_sql'][:60]}...")
    print(f"   Schema: {test_case['schema'][:50]}...")
    print()

## Create Dummy Models for Comparison

In [ ]:
# Create dummy models with different performance characteristics
models = evaluator.create_dummy_models()

print(f"🤖 Created {len(models)} dummy models for comparison:")
for model_name, model in models.items():
    print(f"  - {model_name}: {model.name} (accuracy: {model.accuracy:.1%})")

## Test Individual Metrics

In [ ]:
# Test individual metric calculations
print("🧪 Testing Individual Metrics:")
print("=" * 40)

# Test exact match
sql1 = "SELECT * FROM customers"
sql2 = "SELECT * FROM customers"
sql3 = "SELECT name FROM customers"

exact_match_1 = evaluator._check_exact_match(sql1, sql2)
exact_match_2 = evaluator._check_exact_match(sql1, sql3)

print(f"Exact Match Test:")
print(f"  '{sql1}' vs '{sql2}': {exact_match_1}")
print(f"  '{sql1}' vs '{sql3}': {exact_match_2}")
print()

# Test BLEU score
bleu_score_1 = evaluator._calculate_bleu_score(sql1, sql2)
bleu_score_2 = evaluator._calculate_bleu_score(sql1, sql3)

print(f"BLEU Score Test:")
print(f"  '{sql1}' vs '{sql2}': {bleu_score_1:.3f}")
print(f"  '{sql1}' vs '{sql3}': {bleu_score_2:.3f}")
print()

# Test schema compliance
schema = "customers(customer_id, name, email, region)"
compliant_sql = "SELECT * FROM customers"
non_compliant_sql = "SELECT * FROM orders"

compliance_1 = evaluator._check_schema_compliance(compliant_sql, schema)
compliance_2 = evaluator._check_schema_compliance(non_compliant_sql, schema)

print(f"Schema Compliance Test:")
print(f"  '{compliant_sql}' with schema '{schema}': {compliance_1}")
print(f"  '{non_compliant_sql}' with schema '{schema}': {compliance_2}")

## Run Model Evaluation

In [ ]:
# Run evaluation on all models
print("🔄 Running model evaluation...")
results_df = evaluator.evaluate_models(test_data, models)

print("✅ Evaluation completed!")
print("\n📊 Results:")
print(results_df.round(3))

## Visualize Results

In [ ]:
# Create visualization of results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Model Comparison Results', fontsize=16, fontweight='bold')

# Execution Accuracy
axes[0, 0].bar(results_df['model'], results_df['execution_accuracy'])
axes[0, 0].set_title('Execution Accuracy')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].tick_params(axis='x', rotation=45)

# Exact Match Accuracy
axes[0, 1].bar(results_df['model'], results_df['exact_match_accuracy'])
axes[0, 1].set_title('Exact Match Accuracy')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].tick_params(axis='x', rotation=45)

# BLEU Score
axes[1, 0].bar(results_df['model'], results_df['avg_bleu_score'])
axes[1, 0].set_title('Average BLEU Score')
axes[1, 0].set_ylabel('BLEU Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# Response Time
axes[1, 1].bar(results_df['model'], results_df['avg_response_time'])
axes[1, 1].set_title('Average Response Time')
axes[1, 1].set_ylabel('Time (seconds)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Detailed Performance Analysis

In [ ]:
# Create a comprehensive performance heatmap
metrics_for_heatmap = [
    'execution_accuracy', 'exact_match_accuracy', 
    'schema_compliance_rate', 'avg_bleu_score'
]

heatmap_data = results_df.set_index('model')[metrics_for_heatmap].T

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, cmap='YlOrRd', fmt='.3f', 
            cbar_kws={'label': 'Score'})
plt.title('Model Performance Heatmap', fontsize=14, fontweight='bold')
plt.ylabel('Metrics')
plt.xlabel('Models')
plt.tight_layout()
plt.show()

## Generate Comparison Report

In [ ]:
# Generate detailed comparison report
report = evaluator.generate_comparison_report(results_df)
print("📋 Detailed Comparison Report:")
print("=" * 50)
print(report)

## Model Ranking

In [ ]:
# Create overall ranking based on multiple metrics
# Weight different metrics (you can adjust these weights)
weights = {
    'execution_accuracy': 0.4,
    'exact_match_accuracy': 0.2,
    'schema_compliance_rate': 0.2,
    'avg_bleu_score': 0.2
}

# Calculate weighted score
results_df['weighted_score'] = 0
for metric, weight in weights.items():
    results_df['weighted_score'] += results_df[metric] * weight

# Sort by weighted score
ranking_df = results_df.sort_values('weighted_score', ascending=False)

print("🏆 Model Ranking (Weighted Score):")
print("=" * 40)

for i, (_, row) in enumerate(ranking_df.iterrows(), 1):
    print(f"{i}. {row['model']}")
    print(f"   Weighted Score: {row['weighted_score']:.3f}")
    print(f"   Execution Accuracy: {row['execution_accuracy']:.1%}")
    print(f"   Response Time: {row['avg_response_time']:.3f}s")
    print()

## Error Analysis

In [ ]:
# Analyze error patterns
print("🔍 Error Analysis:")
print("=" * 30)

for _, row in results_df.iterrows():
    model_name = row['model']
    error_rate = row['error_rate']
    
    print(f"{model_name}:")
    print(f"  Error Rate: {error_rate:.1%}")
    print(f"  Successful Queries: {row['total_queries'] - row['errors']}/{row['total_queries']}")
    
    # Performance insights
    if error_rate > 0.2:
        print(f"  ⚠️  High error rate - needs improvement")
    elif error_rate > 0.1:
        print(f"  ⚡ Moderate error rate - room for optimization")
    else:
        print(f"  ✅ Low error rate - good performance")
    
    print()

## Performance vs Accuracy Trade-off

In [ ]:
# Plot performance vs accuracy trade-off
plt.figure(figsize=(10, 6))

scatter = plt.scatter(results_df['avg_response_time'], 
                     results_df['execution_accuracy'],
                     s=results_df['avg_bleu_score'] * 500,  # Size based on BLEU score
                     alpha=0.7,
                     c=range(len(results_df)),
                     cmap='viridis')

# Add model labels
for i, row in results_df.iterrows():
    plt.annotate(row['model'], 
                (row['avg_response_time'], row['execution_accuracy']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=10, alpha=0.8)

plt.xlabel('Average Response Time (seconds)')
plt.ylabel('Execution Accuracy')
plt.title('Performance vs Accuracy Trade-off\n(Bubble size = BLEU Score)')
plt.grid(True, alpha=0.3)

# Add ideal region
plt.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good Accuracy (>80%)')
plt.axvline(x=2.0, color='red', linestyle='--', alpha=0.5, label='Slow Response (>2s)')

plt.legend()
plt.tight_layout()
plt.show()

## Export Results

In [ ]:
# Export results to CSV for further analysis
output_file = 'model_evaluation_results.csv'
results_df.to_csv(output_file, index=False)
print(f"📁 Results exported to {output_file}")

# Show summary statistics
print("\n📈 Summary Statistics:")
print(results_df[['execution_accuracy', 'exact_match_accuracy', 
                  'schema_compliance_rate', 'avg_bleu_score', 
                  'avg_response_time']].describe().round(3))

## Recommendations

In [ ]:
# Generate recommendations based on results
print("💡 Recommendations:")
print("=" * 30)

best_model = results_df.loc[results_df['execution_accuracy'].idxmax()]
fastest_model = results_df.loc[results_df['avg_response_time'].idxmin()]
most_consistent = results_df.loc[results_df['schema_compliance_rate'].idxmax()]

print(f"🏆 Best Overall Accuracy: {best_model['model']} ({best_model['execution_accuracy']:.1%})")
print(f"⚡ Fastest Response: {fastest_model['model']} ({fastest_model['avg_response_time']:.3f}s)")
print(f"🎯 Most Schema Compliant: {most_consistent['model']} ({most_consistent['schema_compliance_rate']:.1%})")

print("\n📋 Action Items:")
if results_df['execution_accuracy'].max() < 0.9:
    print("• Consider fine-tuning models on domain-specific data")
if results_df['avg_response_time'].max() > 3.0:
    print("• Optimize inference pipeline for better response times")
if results_df['schema_compliance_rate'].min() < 0.8:
    print("• Improve schema awareness in model training")

print("\n✅ Evaluation demo completed successfully!")